In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
matA = [[ 3, -1, -1,  0,  0,  0],
        [-1,  4, -1, -1,  0,  0],
        [-1, -1,  4, -1, -1,  0],
        [ 0, -1, -1,  4, -1, -1],
        [ 0,  0, -1, -1,  4, -1],
        [ 0,  0,  0, -1, -1,  3]]

wVec = [5, 5, 0, 0, 0, 0]
np.linalg.solve(matA, wVec)

def banded(N):
    matrix = np.zeros((N, N))
    
    # Lists to hold diagonal and off-diagonal elements
    list1, list2, list3, list4, list5 = [], [], [], [], []
    
    for i in range(N):
        for j in range(N):
            # Diagonal elements
            if i == j:
                if (i == 0 or i == N - 1):
                    matrix[i][j] = 3
                elif (i == 1 or i == N - 2):
                    matrix[i][j] = 4
                else:
                    matrix[i][j] = 4
                    
                # Off-diagonal elements
                if j > 1:
                    matrix[i][j - 2] = -1
                if j > 0:
                    matrix[i][j - 1] = -1
                if j < N - 1:
                    matrix[i][j + 1] = -1
                if j < N - 2:
                    matrix[i][j + 2] = -1
    
    # Collecting elements for banded matrix representation
    for i in range(N):
        for j in range(N):
            if i == j:
                list1.append(matrix[i][j])
            elif i == j - 1:
                list2.append(matrix[i][j])
            elif i == j - 2:
                list3.append(matrix[i][j])
            elif i == j + 1:
                list4.append(matrix[i][j])
            elif i == j + 2:
                list5.append(matrix[i][j])
    
    # Padding zeros to make lists equal in length
    for i in range(len(list1)):
        if len(list2) != len(list1):
            list2.insert(0, 0)
        if len(list3) != len(list1):
            list3.insert(0, 0)
        if len(list4) != len(list1):
            list4.insert(N, 0)
        if len(list5) != len(list1):
            list5.insert(N, 0)
    
    ab = np.array([list3, list2, list1, list4, list5])
    
    W = np.zeros(N)
    W[0] = 5
    W[1] = 5 

    return scipy.linalg.solve_banded((2, 2), ab, W)
    
def direct(N):
    matrix = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
            # Diagonal elements
            if i == j:
                if (i == 0 or i == N - 1):
                    matrix[i][j] = 3
                elif (i == 1 or i == N - 2):
                    matrix[i][j] = 4
                else:
                    matrix[i][j] = 4
                    
                # Off-diagonal elements
                if j > 1:
                    matrix[i][j - 2] = -1
                if j > 0:
                    matrix[i][j - 1] = -1
                if j < N - 1:
                    matrix[i][j + 1] = -1
                if j < N - 2:
                    matrix[i][j + 2] = -1
    
    W = np.zeros(N)
    W[0] = 5
    W[1] = 5
    
    return np.linalg.solve(matrix, W)
    

print(banded(6))
print(direct(6))
print(banded(10000))
print(direct(10000))

In [ ]:
# Load the data
data = np.loadtxt("./data.txt")
data2 = np.loadtxt("./data2.txt")

def leastSquareFit(data):
    # Create the design matrix X
    X = np.column_stack([
        data[:, 0]**2,          # x^2
        data[:, 0]*data[:, 1],  # xy
        data[:, 1]**2,          # y^2
        data[:, 0],             # x
        data[:, 1]              # y
    ])

    bVec = -np.ones(len(data))
    pVec = np.linalg.solve(X.T @ X, -X.T @ bVec)  

    # Unpack pVec into parameters
    A, B, C, D, E = pVec
    print(f'Parameters: A={A}, B={B}, C={C}, D={D}, E={E}')

    # Eigenvalue matrix
    M = np.array([[A, B/2], [B/2, C]])
    eigenvalues, eigenvectors = np.linalg.eig(M)

    # Semi-axes lengths
    a = np.sqrt(1 / np.abs(eigenvalues[0]))
    b = np.sqrt(1 / np.abs(eigenvalues[1]))
    semiMajor = max(a,b)
    semiMinor = min(a,b)
    print(f'Semi-major axis = {semiMajor}')
    print(f'Semi-minor axis = {semiMinor}')

    # Ellipse parameters
    angle = np.degrees(np.arctan2(eigenvectors[1, 0], eigenvectors[0, 0]))
    centerX = -D / (2 * A)
    centerY = -E / (2 * C)
    eccentricity = np.sqrt(1 - semiMinor/semiMajor)
    print(f'eccentricity = {eccentricity}')

    # Plotting
    ellipse = patches.Ellipse([centerX, centerY], 2*a, 2*b, angle=angle, fill=False, color='red', label='Fitted Ellipse')
    plt.gca().add_patch(ellipse)
    plt.scatter(data[:, 0], data[:, 1], label='Data Points')
    plt.legend()
    plt.axis('equal')
    plt.show()

leastSquareFit(data)
leastSquareFit(data2)